```markdown
### GPU Configuration and Memory Management

This code configures TensorFlow to manage GPU resources effectively. It performs the following tasks:

1. **List Available GPUs**:
    - Retrieves a list of physical GPU devices available on the system using `tf.config.experimental.list_physical_devices('GPU')`.

2. **Set Memory Growth**:
    - Enables memory growth for each GPU using `tf.config.experimental.set_memory_growth`. This ensures that TensorFlow only allocates GPU memory as needed, rather than reserving all available memory upfront.

3. **Limit GPU Memory**:
    - Configures virtual devices with a memory limit using `tf.config.experimental.VirtualDeviceConfiguration`. This allows TensorFlow to allocate only a specified amount of GPU memory.

4. **Verification**:
    - Prints the number of physical and logical GPUs detected to confirm the configuration.

This setup is particularly useful when working with multiple GPUs or when running TensorFlow on shared GPU resources.
```

In [ ]:
import tensorflow as tf

# Get list of available GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    try:
        # Set memory growth to False for all GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, False)

        # Limit GPU memory growth to the fraction used by TensorFlow
        # Note: This will allocate the entire GPU memory upfront
        for gpu in gpus:
            physical_device = gpu
            tf.config.experimental.set_virtual_device_configuration(
                physical_device,
                [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=physical_device.memory_limit)]
            )

        # Verification
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)


In [ ]:
import tensorflow as tf

# List all local GPUs detected by TensorFlow
gpus = tf.config.experimental.list_physical_devices('GPU')

# Check if TensorFlow is configured to allocate GPU memory dynamically for each GPU
for gpu in gpus:
    memory_growth = tf.config.experimental.get_memory_growth(gpu)
    print("GPU memory growth for {}: {}".format(gpu.name, memory_growth))


In [ ]:
import numpy as np
import os
import tensorrt
from skimage import io, filters
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

 ### Preprocessing
 (Currently using ISIC 2017)
 
1. **Gaussian Filter for Noise Reduction**: filters.gaussian is applied to the loaded image with a specified sigma value (sigma=5 in this case) to reduce noise.

2. **Gaussian Filter for Smoothing**: Another filters.gaussian filter is applied to the noise-reduced image with a smaller sigma value (sigma=2) to further smooth the image.

3. **Standardization**: The smoothed image is standardized by subtracting the mean value of the image and dividing by its standard deviation. This step helps in normalizing the image intensities.

4. **Resize**: The standardized image is resized to a target size of (224, 224) using cv2.resize. This is commonly done to ensure that all images fed into the model have the same dimensions.

5. **Centering and Normalization**: After resizing, the mean value is subtracted from the resized image, and then the image is divided by twice the standard deviation value. This centers the image around zero and normalizes it. This step ensures that the pixel values are within a certain range, making the model training process more stable.


In [ ]:
def preprocess_images(image_paths):
    preprocessed_images = []
    labels = []
    for image_path in image_paths:
        if os.path.basename(image_path).startswith('.'):
            continue
        
        # Extract label from the folder name
        label = os.path.basename(os.path.dirname(image_path))
        
        original_image = io.imread(image_path)

        # Apply Gaussian filter for noise reduction
        noise_reduced_image = filters.gaussian(original_image, sigma=5)

        # Apply Gaussian filter for smoothing
        smoothed_image = filters.gaussian(noise_reduced_image, sigma=2)

        # Standardize the smoothed image
        mean_value = np.mean(smoothed_image)
        std_value = np.std(smoothed_image)
        standardized_image = (smoothed_image - mean_value) / std_value
        
        # Scale the values to range [0, 1]
        standardized_image = (standardized_image - np.min(standardized_image)) / (np.max(standardized_image) - np.min(standardized_image))

        # Resize the standardized image
        target_size = (224, 224)
        resized_image = cv2.resize(standardized_image, target_size, interpolation=cv2.INTER_AREA)

        # Center and normalize the resized image
        mean_value = np.mean(resized_image)
        std_value = np.std(resized_image)
        centered_normalized_image = (resized_image - mean_value) / (std_value * 2.0)
        
        # Scale the values to range [0, 1]
        centered_normalized_image = (centered_normalized_image - np.min(centered_normalized_image)) / (np.max(centered_normalized_image) - np.min(centered_normalized_image))

        preprocessed_images.append(centered_normalized_image)
        labels.append(label)

    return preprocessed_images, labels

# Set input folders with the path of the melanoma and non melanoma images
input_folders = ['cancer 2018/task_2/akiec', 
                 'cancer 2018/task_2/bcc',
                 'cancer 2018/task_2/bkl',
                 'cancer 2018/task_2/df',
                 'cancer 2018/task_2/mel',
                 'cancer 2018/task_2/nv',
                 'cancer 2018/task_2/vasc']

# Preprocess images
all_images = []
all_labels = []
for folder in input_folders:
    image_paths = [os.path.join(folder, filename) for filename in os.listdir(folder)]
    preprocessed_images, labels = preprocess_images(image_paths)
    all_images.extend(preprocessed_images)
    all_labels.extend(labels)


In [ ]:
import os
import numpy as np
import cv2
from skimage import io, filters
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from skimage.metrics import structural_similarity, mean_squared_error, peak_signal_noise_ratio
# Preprocessing function to generate noisy and reconstructed images
def preprocess_single_image(image_path):
    original_image = io.imread(image_path)
    # Resize the standardized image
    target_size = (224, 224)
    resized_image = cv2.resize(original_image, target_size, interpolation=cv2.INTER_AREA)

    # Apply Gaussian filter for noise reduction
    noise_reduced_image = filters.gaussian(resized_image, sigma=5)

    # Apply Gaussian filter for smoothing
    smoothed_image = filters.gaussian(noise_reduced_image, sigma=2)

    # Standardize the smoothed image
    mean_value = np.mean(smoothed_image)
    std_value = np.std(smoothed_image)
    standardized_image = (smoothed_image - mean_value) / std_value

    

    # Center and normalize the resized image
    mean_value = np.mean(standardized_image)
    std_value = np.std(standardized_image)
    centered_normalized_image = (standardized_image - mean_value) / (std_value * 2.0)

    # Scale the values to range [0, 1]
    centered_normalized_image = (centered_normalized_image - np.min(centered_normalized_image)) / (np.max(centered_normalized_image) - np.min(centered_normalized_image))

    return resized_image, smoothed_image

noisy_image, centered_normalized_image = preprocess_single_image('cancer 2017/melanoma/ISIC_ISIC_0000148.jpg')

print('ssim',structural_similarity(noisy_image, centered_normalized_image,channel_axis=2))

print('mse',mean_squared_error(noisy_image, centered_normalized_image)/1000)

print('psnr',peak_signal_noise_ratio(noisy_image, centered_normalized_image))
#print(calculate_ssim(noisy_image, centered_normalized_image))

Since preprocessing 2000 images takes more than an hour, it's saved into a numpy array, which is loaded later. This way, we only need to perform the preprocessing once.

In [ ]:
# Save preprocessed images to a file
np.save('all_images_1000.npy', all_images)
np.save('all_labels_1000.npy',all_labels)

Later on we will be only loading these array for the images and labels so that it can be used for training

In [ ]:
# Loading the numpy array into all_images and all_labels
all_images = np.load('images_aug.npy')
all_labels = np.load('labels-aug.npy')
print(all_images.shape)
print(all_labels.shape)